In [1]:
# LIBRARY
## Essentials
import array
import pandas as pd
import numpy as np

## Regression
import statsmodels.formula.api as smf

## Plotting
import matplotlib.pyplot as plt
import seaborn; seaborn.set() # set plot style

In [2]:
# DATA
LL = pd.read_stata('..\data\LL_train.dta')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 10)

SE = pd.read_stata('..\data\self_employment.dta')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10)

In [3]:
# MANIPULATION

## Create income per million variable
## # data['newvar'] = data.oldvar / 1000000
LL['income_mill'] = LL.income_month/1000000

## Create new quadratic variable
## data['newvar'] = data.oldvar ** 2
SE['agesqr'] = SE.age ** 2

In [4]:
# OLS REGRESSION
# MODEL = smf.ols('DEPENDENT ~ INDEPENDENT', data=DATA).fit()

## Y = income_month
### OLS Regression
ols1 = smf.ols('income_month ~ age + agesqr + female', data=LL).fit()

### OLS Regression with quadratic variable (assuming agesqr is not exist)
ols2 = smf.ols('income_month ~ age + I(age**2) + female', data=LL).fit()


## Y = income_mill
#### OLS Regression
ols3 = smf.ols('income_mill ~ age + agesqr + female', data=LL).fit()

#### OLS Regression with quadratic variable (assuming agesqr is not exist)
ols4 = smf.ols('income_mill ~ age + I(age**2) + female', data=LL).fit()

In [5]:
# RESULT 
## [income_month]
print(ols1.summary())
print(ols2.summary())

## [income_mill]
print(ols3.summary())
print(ols4.summary())

                            OLS Regression Results                            
Dep. Variable:           income_month   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     97.45
Date:                Fri, 18 Jan 2019   Prob (F-statistic):           2.61e-62
Time:                        20:18:58   Log-Likelihood:            -1.9563e+05
No. Observations:               11914   AIC:                         3.913e+05
Df Residuals:                   11910   BIC:                         3.913e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.768e+06   2.62e+05     -6.762      0.0

In [10]:
# Probit Regression
# MODELNAME = smf.probit('DEPENDENT ~ INDEPENDENT', data=DATA).fit()
probit1 = smf.probit('selfemployed ~ age + agesqr + female + tertiary_educ', data=SE).fit()

# Logit Regression
# MODELNAME = smf.logit('DEPENDENT ~ INDEPENDENT', data=DATA).fit()
logit1 = smf.logit('selfemployed ~ age + agesqr + female + tertiary_educ', data=SE).fit()

Optimization terminated successfully.
         Current function value: 0.512647
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.512927
         Iterations 6


In [11]:
# RESULT
## Probit
print(probit1.summary())

## Logit
print(logit1.summary())

                          Probit Regression Results                           
Dep. Variable:           selfemployed   No. Observations:                86120
Model:                         Probit   Df Residuals:                    86115
Method:                           MLE   Df Model:                            4
Date:                Fri, 18 Jan 2019   Pseudo R-squ.:                  0.1273
Time:                        20:21:04   Log-Likelihood:                -44149.
converged:                       True   LL-Null:                       -50592.
                                        LLR p-value:                     0.000
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -3.1046      0.033    -93.139      0.000      -3.170      -3.039
age               0.1200      0.002     76.813      0.000       0.117       0.123
agesqr           -0.0011   1.69e-05    -

In [12]:
## Marginal effect
## ref: https://www.statsmodels.org/dev/generated/statsmodels.discrete.discrete_model.LogitResults.get_margeff.html#statsmodels.discrete.discrete_model.LogitResults.get_margeff
## MODEL.get_margeff(at='OPTION', method='OPTION', atexog='OPTION, dummy='OPTION', count='OPTION').summary()

### [Probit]
probit1.get_margeff(at='mean', method='dydx', atexog=None, dummy=True, count=False).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:           selfemployed
Method:                          dydx
At:                              mean
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age               0.0372      0.000     80.104      0.000       0.036       0.038
agesqr           -0.0003   5.07e-06    -67.132      0.000      -0.000      -0.000
female           -0.1514      0.003    -50.070      0.000      -0.157      -0.146
tertiary_educ    -0.1457      0.004    -33.945      0.000      -0.154      -0.137
=================================================================================
"""

In [13]:
### [Logit]
logit1.get_margeff(at='mean', method='dydx', atexog=None, dummy=True, count=False).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:           selfemployed
Method:                          dydx
At:                              mean
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age               0.0370      0.000     79.465      0.000       0.036       0.038
agesqr           -0.0003    5.1e-06    -66.626      0.000      -0.000      -0.000
female           -0.1493      0.003    -50.252      0.000      -0.155      -0.144
tertiary_educ    -0.1420      0.004    -35.559      0.000      -0.150      -0.134
=================================================================================
"""